## Check GPU availability

In [1]:
# pip install torch tensorflow
# pip install boto3 -q
# pip install s3cmd -q
import torch
import os
import boto3

In [2]:

# Check if CUDA (GPU support for PyTorch) is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    gpu_count = torch.cuda.device_count()
    print(f"Found {gpu_count} GPU(s) available.")
    for i in range(gpu_count):
        print(f"GPU is : {torch.cuda.get_device_name(i)}")
else:
    print("No GPU available.")


Found 1 GPU(s) available.
GPU 0: NVIDIA A10G


### Check connection to s3 storage

In [2]:
# Get environment variables
import os
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_default_region = None
aws_s3_bucket = os.getenv("AWS_S3_BUCKET")
aws_s3_endpoint = 'http://s3.openshift-storage.svc:80/'

print(aws_s3_bucket)

llm-6219e3b5-aae8-4b37-852c-6fcd7180339a



#### 1. boto3 client

In [3]:
# Create S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_default_region,
    endpoint_url=aws_s3_endpoint,
    use_ssl=False
)

# Test the connection by listing buckets
try:
    response = s3.list_buckets()
    print("Connection successful. Available buckets:")
    for bucket in response['Buckets']:
        print(bucket['Name'])
except Exception as e:
    print(f"Connection failed: {e}")

NameError: name 'boto3' is not defined

#### 2: Minio CLI

In [20]:
!curl https://dl.min.io/client/mc/release/linux-amd64/mc --create-dirs -o ./minio-binaries/mc
!chmod +x ./minio-binaries/mc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.2M  100 25.2M    0     0  16.6M      0  0:00:01  0:00:01 --:--:-- 16.6M


In [21]:
!./minio-binaries/mc alias set noobaa $AWS_S3_ENDPOINT --api "S3v4" $AWS_ACCESS_KEY_ID $AWS_SECRET_ACCESS_KEY

]11;?\mc: Configuration written to `/opt/app-root/src/.mc/config.json`. Please update your access credentials.
mc: Successfully created `/opt/app-root/src/.mc/share`.
mc: Initialized share uploads `/opt/app-root/src/.mc/share/uploads.json` file.
mc: Initialized share downloads `/opt/app-root/src/.mc/share/downloads.json` file.
Added `noobaa` successfully.


In [22]:
!./minio-binaries/mc alias ls

]11;?\gcs   
  URL       : https://storage.googleapis.com
  AccessKey : YOUR-ACCESS-KEY-HERE
  SecretKey : YOUR-SECRET-KEY-HERE
  API       : S3v2
  Path      : dns

local 
  URL       : http://localhost:9000
  AccessKey : 
  SecretKey : 
  API       : 
  Path      : auto

noobaa
  URL       : http://s3.openshift-storage.svc:80
  AccessKey : XV07CKsPMJTOW2MDMmBt
  SecretKey : +yD7+DX6feq3M12GNrlE6LTU+GHBPK8tHXZUsaau
  API       : S3v4
  Path      : auto

play  
  URL       : https://play.min.io
  AccessKey : Q3AM3UQ867SPQQA43P2F
  SecretKey : zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG
  API       : S3v4
  Path      : auto

s3    
  URL       : https://s3.amazonaws.com
  AccessKey : YOUR-ACCESS-KEY-HERE
  SecretKey : YOUR-SECRET-KEY-HERE
  API       : S3v4
  Path      : dns



In [23]:
!./minio-binaries/mc alias ls noobaa

]11;?\noobaa
  URL       : http://s3.openshift-storage.svc:80
  AccessKey : XV07CKsPMJTOW2MDMmBt
  SecretKey : +yD7+DX6feq3M12GNrlE6LTU+GHBPK8tHXZUsaau
  API       : S3v4
  Path      : auto



#### 3: s3cmd

In [12]:
pip install s3cmd -q


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
!s3cmd --access_key=$AWS_ACCESS_KEY_ID --secret_key=$AWS_SECRET_ACCESS_KEY --host=$AWS_S3_ENDPOINT --no-ssl ls


2024-05-03 03:21  s3://llm-6219e3b5-aae8-4b37-852c-6fcd7180339a


In [10]:
s3c="s3cmd --access_key=$AWS_ACCESS_KEY_ID --secret_key=$AWS_SECRET_ACCESS_KEY --host=$AWS_S3_ENDPOINT --no-ssl"
!$s3c ls

2024-05-04 13:46  s3://llm-6219e3b5-aae8-4b37-852c-6fcd7180339a
